In [22]:
import xarray
import rioxarray
from pystac_client import Client
import geopandas
import planetary_computer

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [15]:
time_range = "2022-09-01/2022-09-30"
area_of_interest = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -123.00253539948329,
              49.836668567525265
            ],
            [
              -123.00253539948329,
              49.80080680843781
            ],
            [
              -122.91329388295358,
              49.80080680843781
            ],
            [
              -122.91329388295358,
              49.836668567525265
            ],
            [
              -123.00253539948329,
              49.836668567525265
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

search = catalog.search(collections=["landsat-c2-l2"], intersects=area_of_interest["features"][0]["geometry"], datetime=time_range)
items = search.get_all_items()
len(items)

6

In [11]:
df = geopandas.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
df

,geometry,gsd,created,sci:doi,datetime,platform,proj:epsg,proj:shape,description,instruments,...,landsat:wrs_row,landsat:scene_id,landsat:wrs_path,landsat:wrs_type,view:sun_azimuth,landsat:correction,view:sun_elevation,landsat:cloud_cover_land,landsat:collection_number,landsat:collection_category
0,"POLYGON ((-123.15685 51.34277, -123.84151 49.6...",30,2022-10-19T09:17:23.148339Z,10.5066/P9OGBGM6,2022-09-29T19:01:23.824034Z,landsat-9,32610,"[7931, 7851]",Landsat Collection 2 Level-2,"[oli, tirs]",...,025,LC90470252022272LGN00,047,2,162.147720,L2SP,35.654826,69.69,02,T1
1,"POLYGON ((-124.71529 51.34407, -125.39905 49.6...",30,2022-10-19T09:17:26.279232Z,10.5066/P9OGBGM6,2022-09-28T19:07:51.660162Z,landsat-8,32610,"[8051, 7951]",Landsat Collection 2 Level-2,"[oli, tirs]",...,025,LC80480252022271LGN00,048,2,162.027544,L2SP,36.030519,93.26,02,T1
2,"POLYGON ((-123.17452 51.34463, -123.85899 49.6...",30,2022-10-27T09:23:53.182063Z,10.5066/P9OGBGM6,2022-09-21T19:01:41.204186Z,landsat-8,32610,"[7951, 7851]",Landsat Collection 2 Level-2,"[oli, tirs]",...,025,LC80470252022264LGN01,047,2,160.581130,L2SP,38.576009,2.19,02,T1
3,"POLYGON ((-123.14996 51.34251, -123.83486 49.6...",30,2022-09-19T09:34:56.080538Z,10.5066/P9OGBGM6,2022-09-13T19:01:18.734720Z,landsat-9,32610,"[7931, 7851]",Landsat Collection 2 Level-2,"[oli, tirs]",...,025,LC90470252022256LGN00,047,2,158.680818,L2SP,41.415433,26.98,02,T1
4,"POLYGON ((-123.15944 51.34492, -123.84445 49.6...",30,2022-09-17T09:18:09.985651Z,10.5066/P9OGBGM6,2022-09-05T19:01:35.638237Z,landsat-8,32610,"[7951, 7851]",Landsat Collection 2 Level-2,"[oli, tirs]",...,025,LC80470252022248LGN00,047,2,156.848237,L2SP,44.194336,32.11,02,T1
5,"POLYGON ((-124.69283 51.34251, -125.37695 49.6...",30,2022-09-17T09:18:15.401393Z,10.5066/P9OGBGM6,2022-09-04T19:07:26.772365Z,landsat-9,32610,"[8031, 7951]",Landsat Collection 2 Level-2,"[oli, tirs]",...,025,LC90480252022247LGN00,048,2,156.500903,L2SP,44.510751,89.92,02,T1


In [4]:
selected_item = min(items, key=lambda item: item.properties["eo:cloud_cover"])
selected_item

ID: LC08_L2SP_047025_20220921_02_T1
"Bounding Box: [-123.9206158, 49.18230516, -120.53971475, 51.34789484]"
Datetime: 2022-09-21 19:01:41.204186+00:00
gsd: 30
created: 2022-10-27T09:23:53.182063Z
sci:doi: 10.5066/P9OGBGM6
datetime: 2022-09-21T19:01:41.204186Z
platform: landsat-8
proj:epsg: 32610
"proj:shape: [7951, 7851]"
description: Landsat Collection 2 Level-2


In [5]:
import rich.table

table = rich.table.Table("Asset Key", "Descripiption")
for asset_key, asset in selected_item.assets.items():
    # print(f"{asset_key:<25} - {asset.title}")
    table.add_row(asset_key, asset.title)

table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Descripiption                                                        ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ qa               │ Surface Temperature Quality Assessment Band                          │
│ ang              │ Angle Coefficients File                                              │
│ red              │ Red Band                                                             │
│ blue             │ Blue Band                                                            │
│ drad             │ Downwelled Radiance Band                                             │
│ emis             │ Emissivity Band                                                      │
│ emsd             │ Emissivity Standard Deviation Band                                   │
│ trad             │ Thermal Radiance Band                                                │
│ urad             │ Upwelled Radiance Band                                               │
│ atran            │ Atmospheric Transmittance Band                                       │
│ cdist            │ Cloud Distance Band                                                  │
│ green            │ Green Band                                                           │
│ nir08            │ Near Infrared Band 0.8                                               │
│ lwir11           │ Surface Temperature Band                                             │
│ swir16           │ Short-wave Infrared Band 1.6                                         │
│ swir22           │ Short-wave Infrared Band 2.2                                         │
│ coastal          │ Coastal/Aerosol Band                                                 │
│ mtl.txt          │ Product Metadata File (txt)                                          │
│ mtl.xml          │ Product Metadata File (xml)                                          │
│ mtl.json         │ Product Metadata File (json)                                         │
│ qa_pixel         │ Pixel Quality Assessment Band                                        │
│ qa_radsat        │ Radiometric Saturation and Terrain Occlusion Quality Assessment Band │
│ qa_aerosol       │ Aerosol Quality Assessment Band                                      │
│ tilejson         │ TileJSON with default rendering                                      │
│ rendered_preview │ Rendered preview                                                     │
└──────────────────┴──────────────────────────────────────────────────────────────────────┘

In [33]:
from IPython.display import Image

Image(url=selected_item.assets["rendered_preview"].href, width=500)

In [20]:
signed_assets = {"red": planetary_computer.sign(selected_item).assets["red"],
                 "green": planetary_computer.sign(selected_item).assets["green"],
                 "blue": planetary_computer.sign(selected_item).assets["blue"],
                 "swir": planetary_computer.sign(selected_item).assets["swir16"],
                 "nir": planetary_computer.sign(selected_item).assets["nir08"]}

In [23]:
ds = None
for key in signed_assets:
    if ds is None:
        ds = rioxarray.open_rasterio(signed_assets[key].href, overview_level=4).squeeze()
        ds.name = key

    band_ds = rioxarray.open_rasterio(signed_assets[key].href, overview_level=4).squeeze()
    band_ds.name = key
    ds = xarray.merge([ds, band_ds])

In [ ]:
ds["ndsi"] = (ds["green"] - ds["swir"]) / (ds["green"] + ds["swir"])
img = ds["ndsi"].plot(cmap="Greys", add_colorbar=False)
img.axes.set_axis_off();

In [57]:
from geocube.api.core import make_geocube

out_g = make_geocube(vector_data="g.geojson", resolution=(-0.0001, 0.0001))
out_g

<xarray.Dataset>
Dimensions:      (y: 359, x: 894)
Coordinates:
  * y            (y) float64 49.84 49.84 49.84 49.84 ... 49.8 49.8 49.8 49.8
  * x            (x) float64 -123.0 -123.0 -123.0 ... -122.9 -122.9 -122.9
    spatial_ref  int64 0
Data variables:
    value        (y, x) float64 nan 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 nan

In [66]:
a, b = xarray.align(out_g, ds)

<xarray.Dataset>
Dimensions:      (y: 0, x: 0)
Coordinates:
  * y            (y) float64 
  * x            (x) float64 
    band         int64 1
    spatial_ref  int64 0
Data variables:
    red          (y, x) uint16 
    green        (y, x) uint16 
    blue         (y, x) uint16 
    swir         (y, x) uint16 
    nir          (y, x) uint16 
Attributes:
    AREA_OR_POINT:  Point
    _FillValue:     0
    scale_factor:   1.0
    add_offset:     0.0

IndexError: index -1 is out of bounds for axis 0 with size 0